In [ ]:
#| default_exp evaluation

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from wxbs_benchmark.dataset import *
from wxbs_benchmark.metrics import *

def evaluate_Fs(Fs = [], subset = 'test'):
    dset = WxBSDataset('.WxBS', subset=subset, download=True)
    ths = np.arange(20)
    gt_corrs = []
    names = []
    for data_dict in dset:
        corrs = data_dict['pts']
        pairname = data_dict['name']
        names.append(pairname)
        gt_corrs.append(corrs)
    assert len(Fs) == len(gt_corrs)
    per_pair_results = {}
    all_res = []
    for (F, pts, pairname) in zip(Fs, gt_corrs, names):
        res = fraction_of_gt_corrs_consisent_with_F(F, pts, ths)
        per_pair_results[pairname] = res
        all_res.append(res)
    per_pair_results['average'] = np.stack(all_res, axis=1).mean(axis=1)
    return per_pair_results, ths

Evaluates fundamental matrices returned by algorithm

In [ ]:
shift_f = np.array([[0.0, 0.0, 0.0],
              [0.0, 0.0, -1.0],
              [0.0, 1.0, 0.0]])
Fs = [shift_f, shift_f]

res_dict, th = evaluate_Fs(Fs, 'val')
print (res_dict)

{'WGABS/petrzin': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.02631579, 0.05263158, 0.05263158], 'WGALBS/kyiv_dolltheater2': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'average': array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01315789, 0.01315789,
       0.01315789, 0.01315789, 0.01315789, 0.02631579, 0.02631579],
      dtype=float32)}


In [ ]:
#| export
def evaluate_corrs(estimated_right = [],
                   estimated_left = [],
                   subset = 'test'):
    dset = WxBSDataset('.WxBS', subset=subset, download=True)
    ths = np.arange(20)
    gt_corrs = []
    names = []
    for data_dict in dset:
        corrs = data_dict['pts']
        pairname = data_dict['name']
        names.append(pairname)
        gt_corrs.append(corrs)
    assert len(estimated_right) == len(gt_corrs)
    assert len(estimated_left) == len(gt_corrs)
    all_res = []
    per_pair_results = {}
    for (est_right, est_left, gt_pts, pairname) in zip(estimated_right,
                                                       estimated_left, 
                                                       gt_corrs, names):
        res = 0.5 * (PCK(est_right, gt_pts[:, 2:4], ths) + 
                     PCK(est_left,  gt_pts[:, :2], ths))
        per_pair_results[pairname] = res
        all_res.append(res)
    per_pair_results['average'] = np.stack(all_res, axis=1).mean(axis=1)
    return per_pair_results, ths

We will check the naive baseline -- no camera motion, return the input baseline.

In [ ]:
dset = WxBSDataset('.WxBS', subset='val', download=True)
predicted_left = []
predicted_right = []
for res_dict in dset:
    gt_corrs = res_dict['pts']
    query_left = gt_corrs[:, :2]
    query_right = gt_corrs[:, 2:]
    predicted_right.append(query_left)
    predicted_left.append(query_right)
eval_results = evaluate_corrs (predicted_right, predicted_left, subset='val')
print (eval_results)

({'WGABS/petrzin': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32), 'WGALBS/kyiv_dolltheater2': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32), 'average': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)}, array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19]))


In [ ]:
#| export
def evaluate_Hs(Hs = []):
    dset = EVDDataset('.EVD', download=True)
    ths = np.logspace(np.log2(1.0), np.log2(20), 10, base=2.0)
    gt_homos = []
    names = []
    shapes1 = []
    shapes2 = []
    
    for data_dict in dset:
        gt_homo = data_dict['H']
        pairname = data_dict['name']
        shapes1.append(data_dict['img1_shape'])
        shapes2.append(data_dict['img2_shape'])
        names.append(pairname)
        gt_homos.append(gt_homo)
    assert len(Hs) == len(gt_homos)
    per_pair_results = {}
    all_res = []
    for (H, Hgt, sh1, sh2, pairname) in zip(Hs, gt_homos, shapes1, shapes2, names):
        mae = get_visible_part_mean_absolute_reprojection_error(sh1, sh2, Hgt, H)
        per_pair_results[pairname] = mae
        all_res.append(mae)
        
    per_pair_results['average'] = get_mAA(all_res, ths)
    return per_pair_results, ths

In [ ]:
dset = EVDDataset('.EVD', download=True)
predicted_Hs = []
for res_dict in dset:
    gt_H = res_dict['H']
    predicted_Hs.append(gt_H)
eval_results = evaluate_Hs (predicted_Hs)
print (eval_results)

({'adam': 0.0, 'cafe': 0.0, 'cat': 0.0, 'dum': 0.0, 'face': 0.0, 'fox': 0.0, 'girl': 0.0, 'graf': 0.0, 'grand': 0.0, 'index': 0.0, 'mag': 0.0, 'pkk': 0.0, 'shop': 0.0, 'there': 0.0, 'vin': 0.0, 'average': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, array([ 1.        ,  1.39495079,  1.94588772,  2.71441762,  3.78647901,
        5.2819519 ,  7.368063  , 10.27808533, 14.33742329, 20.        ]))
